<a href="https://colab.research.google.com/github/kaispace30098/Recommendation_system_MF_Deep_Learning_modeling/blob/main/Recommendation_System_Matrix_Fatorization_Deep_Learning_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import tensorflow as tf

In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dot, Add, Flatten,Concatenate,Dense,Activation,BatchNormalization,Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD

In [4]:
def read_dir_file(case_f):  # case_f = 0 for uploading one File and case_f = 1 for uploading one Zipped Directory
        #uploaded = files.upload()    # to upload a Full Directory, please Zip it first (use WinZip)
        for fn in uploaded.keys():
            name = fn  #.encode('utf-8')
            #print('\nfile after encode', name)
            #name = io.BytesIO(uploaded[name])
        if case_f == 0:    # case of uploading 'One File only'
            print('\n file name: ', name)
            return name
        else:   # case of uploading a directory and its subdirectories and files
            zfile = zipfile.ZipFile(name, 'r')   # unzip the directory 
            zfile.extractall()
            for d in zfile.namelist():   # d = directory
                print('\n main directory name: ', d)
                return d

import io
from google.colab import files
uploaded = files.upload()
fileName = read_dir_file(0)
df=pd.read_csv(io.BytesIO(uploaded[fileName]),encoding='utf-8')

Saving smallmovie_rating.csv to smallmovie_rating.csv

 file name:  smallmovie_rating.csv


In [5]:
df.columns

Index(['Unnamed: 0', 'userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [6]:
N=df.userId.max()+1
M=df.movieId.max()+1

df=shuffle(df)
cutoff=int(0.8*len(df))
df_train=df.iloc[:cutoff]
df_test=df.iloc[cutoff:]

In [7]:
#initialize variables
K=9 #latent dimensionality
mu=df_train.rating.mean()
epoch=20
reg=0.

In [8]:
#Model 1# Matrix factorization
u=Input(shape=(1,))
m=Input(shape=(1,))

u_embedding=Embedding(N,K,embeddings_regularizer=l2(reg))(u)
m_embedding=Embedding(M,K,embeddings_regularizer=l2(reg))(m)

u_bias=Embedding(N,1,embeddings_regularizer=l2(reg))(u)
m_bias=Embedding(M,1,embeddings_regularizer=l2(reg))(m)
x=Dot(axes=2)([u_embedding,m_embedding])
x=Add()([x,u_bias,m_bias])
x=Flatten()(x)
model=Model(inputs=[u,m],outputs=x)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1, 9)         90000       ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 1, 9)         18000       ['input_2[0][0]']                
                                                                                              

In [9]:
model.compile(
    loss='mse',
    optimizer=SGD(lr=0.01,momentum=0.9),
    metrics=['mse'])

r=model.fit(x=[df_train.userId.values,df_train.movieId.values],
            y=df_train.rating.values-mu,
            epochs=epoch,
            batch_size=128,
            validation_data=([df_test.userId.values,df_test.movieId.values],
            df_test.rating.values-mu),
            callbacks= tf.keras.callbacks.TensorBoard(log_dir='logs/matrix factorization')
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/20
33701/33701 [==============================] - 85s 2ms/step - loss: 0.7832 - mse: 0.7832 - val_loss: 0.7092 - val_mse: 0.7092
Epoch 2/20
33701/33701 [==============================] - 84s 2ms/step - loss: 0.6923 - mse: 0.6923 - val_loss: 0.6813 - val_mse: 0.6813
Epoch 3/20
33701/33701 [==============================] - 84s 2ms/step - loss: 0.6765 - mse: 0.6765 - val_loss: 0.6736 - val_mse: 0.6736
Epoch 4/20
33701/33701 [==============================] - 83s 2ms/step - loss: 0.6716 - mse: 0.6716 - val_loss: 0.6712 - val_mse: 0.6712
Epoch 5/20
33701/33701 [==============================] - 84s 2ms/step - loss: 0.6699 - mse: 0.6699 - val_loss: 0.6704 - val_mse: 0.6704
Epoch 6/20
33701/33701 [==============================] - 83s 2ms/step - loss: 0.6692 - mse: 0.6692 - val_loss: 0.6700 - val_mse: 0.6700
Epoch 7/20
33701/33701 [==============================] - 86s 3ms/step - loss: 0.6689 - mse: 0.6689 - val_loss: 0.6699 - val_mse: 0.6699
Epoch 8/20
33701/33701 [=================

In [10]:
#Model 2 Deep Nueral Networks
u=Input(shape=(1,))
m=Input(shape=(1,))

u_embedding=Embedding(N,K)(u)
m_embedding=Embedding(M,K)(m)
u_embedding=Flatten()(u_embedding)
m_embedding=Flatten()(m_embedding)
x=Concatenate()([u_embedding,m_embedding])

x=Dense(400,activation='relu')(x)
x=Dropout(0.3)(x)
x=BatchNormalization()(x)
x=Dense(100,activation='relu')(x)
x=Dropout(0.3)(x)
x=BatchNormalization()(x)
x=Dense(1)(x)

model2=Model(inputs=[u,m],outputs=x)

In [11]:
model2.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 1, 9)         90000       ['input_3[0][0]']                
                                                                                                  
 embedding_5 (Embedding)        (None, 1, 9)         18000       ['input_4[0][0]']                
                                                                                            

In [12]:
model2.compile(
    loss='mse',
    optimizer=SGD(lr=0.01,momentum=0.9),
    metrics=['mse'])

r2=model2.fit(x=[df_train.userId.values,df_train.movieId.values],
            y=df_train.rating.values-mu,
            epochs=epoch,
            batch_size=128,
            validation_data=([df_test.userId.values,df_test.movieId.values],
            df_test.rating.values-mu),
            callbacks=tf.keras.callbacks.TensorBoard(log_dir='logs/deep NN')
)


Epoch 1/20


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


33701/33701 [==============================] - 86s 3ms/step - loss: 0.6959 - mse: 0.6959 - val_loss: 0.6503 - val_mse: 0.6503
Epoch 2/20
33701/33701 [==============================] - 85s 3ms/step - loss: 0.6338 - mse: 0.6338 - val_loss: 0.6271 - val_mse: 0.6271
Epoch 3/20
33701/33701 [==============================] - 85s 3ms/step - loss: 0.6234 - mse: 0.6234 - val_loss: 0.6171 - val_mse: 0.6171
Epoch 4/20
33701/33701 [==============================] - 83s 2ms/step - loss: 0.6137 - mse: 0.6137 - val_loss: 0.6122 - val_mse: 0.6122
Epoch 5/20
33701/33701 [==============================] - 84s 2ms/step - loss: 0.6084 - mse: 0.6084 - val_loss: 0.6077 - val_mse: 0.6077
Epoch 6/20
33701/33701 [==============================] - 85s 3ms/step - loss: 0.6025 - mse: 0.6025 - val_loss: 0.6010 - val_mse: 0.6010
Epoch 7/20
33701/33701 [==============================] - 85s 3ms/step - loss: 0.5925 - mse: 0.5925 - val_loss: 0.5953 - val_mse: 0.5953
Epoch 8/20
33701/33701 [============================

In [13]:
!tensorboard dev upload --logdir ./logs/ \
  --name "matrix factorization"\
  --description "Use Keras' Embedding layers to implement Matrix Factorization."\
  --one_shot

2022-05-27 04:02:47.189403: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

./logs/

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) y

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=373649185512-8v619h5kft38l4456nm2dj4ubeqsrvh6.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth

In [37]:
#Let's take a close look at use User_number
User_number=1 #@param {type:"number"}
listm=[]
user1df=df[df.userId==User_number].sort_values(['movieId'])
for i in range(max(user1df.movieId.tolist())):
  if i not in user1df.movieId.tolist():
    listm.append(i)

listu=[1]* len(listm)
pred=[]
for i in range(1,len(listm)):
  pred.append(model2.predict([np.array([listu[i-1]]),np.array([listm[i-1]])]))


max_item = max(pred)
listm[[index for index in range(len(pred)) if pred[index] == max_item][0]]
print(f'The most recommended movie for user {User_number} is movie',
        listm[[index for index in range(len(pred)) if pred[index] == max_item][0]])


The most recommended movie for user 1 is movie 599
